In [5]:
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
from PIL import Image
import urllib.request
import splitfolders
import pandas as pd
import requests
import random
import time
import PIL
import cv2
import os



In [2]:
def save_photos(df):  
    for ind in df.index:
        image_url_list = df['Image_url'][ind]
        stone_name = df['Name'][ind]
        if(stone_name.find(' Opal') != -1 or stone_name.find(' Diamond') != -1 or stone_name.find(" Bone") != -1):continue
        FilePath = 'Data/'
        for i,image_url in enumerate(image_url_list):
            newpath = FilePath + stone_name +'/'
            if not os.path.exists(newpath): os.makedirs(newpath)
            image_url = 'https://www.mindat.org/'+image_url
            response = requests.get(image_url)
            image_name = stone_name +'_num_'+str(i) + '.png' 
            with open(newpath +image_name, 'wb') as file:
                file.write(response.content)
            img = cv2.imread(image_name)


def get_photos(driverf,stone_name):
    time.sleep(1)   
    img_list = []
    try:
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE)
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(stone_name)
        time.sleep(1)
        driverf.find_element_by_css_selector("input[id='searchbox']").send_keys(Keys.ENTER)

        #SCROLL DOWN
        SCROLL_PAUSE_TIME = 1.3
        last_height = driverf.execute_script("return document.body.scrollHeight")
        while True:
            driverf.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height =  driverf.execute_script("return document.body.scrollHeight")
            if(new_height==last_height): break
            last_height = new_height
    except ElementClickInterceptedException:
        return img_list
    except NoSuchElementException:
        pass
    except TypeError: pass
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    try:
        table = soup.find("div",{"id":"mainwrap"})
        images_table = table.find("div",{"id":"photoscroll"})
        images = images_table.find_all("a")

        for img in images:
                img_list.append(img.find('img')['src'])
    except TypeError: pass
    except NoSuchElementException: pass
    except AttributeError: pass
    return img_list

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driverf = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://geology.com/gemstones/') #Gets the most popular gemstone's names
driverf.get('https://www.mindat.org/photoscroll.php') # Gets the gemstone's images

soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find("div",{"class":"right"})
Stones_list = []

href_list = table.find_all('a')
driver.close()
for href in href_list:
    time.sleep(1)
    data_dict = {}
    Name = href.text
    if(Name.find("K2 Granite")!=-1): Name = "Granite"
    if(Name.find(' Opal') != -1 or Name.find(' Diamond') != -1 or Name.find("Bone") != -1):continue
    data_dict['Name'] = Name
    image_list = get_photos(driverf, Name)
    image_list = list(dict.fromkeys(image_list))
    data_dict['Image_url'] = image_list
    Stones_list.append(data_dict)


driverf.close()    



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\noamb\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
C:\Users\noamb\AppData\Local\Temp/ipykernel_10920/2539929547.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST chromedriver version for 95.0.4638 google-chrome
Driver [C:\Users\noamb\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
C:\Users\noamb\AppData\Local\Temp/ipykernel_10920/2539929547.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driverf = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\noamb\AppData\Local\Temp/ipykernel_10920/2513914088.py:22: DeprecationWarning: f

In [4]:
df = pd.DataFrame(Stones_list)
save_photos(df)

In [5]:
df['Name'].to_csv('Stones_Labels.csv')

In [6]:
df

,Name,Image_url
0,Amazonite,[/imagecache/a4/1f/09067590016368433599249.jpg...
1,Amethyst,[/imagecache/61/c8/00885170016366989995197.jpg...
2,Ametrine,[/imagecache/20/6c/05932610016333027797050.jpg...
3,Ammolite,[/imagecache/2f/bd/04625920016287387465282.jpg...
4,Andalusite,[/imagecache/fc/e3/06366350016368435696921.jpg...
...,...,...
109,Variscite,[/imagecache/bb/60/05751940016367274059304.jpg...
110,Vesuvianite,[/imagecache/a4/68/08874840016369265614387.jpg...
111,Wonderstone,[/imagecache/e4/f1/02164130016364600649624.jpg...
112,Zircon,[/imagecache/f9/45/01820660016359153653711.jpg...


<h1> Split the data </h1>

In [6]:
input_folder = 'Data_Sample'
output_folder = 'Splited_Data_Sample'
splitfolders.ratio(input_folder, output_folder,
                   seed=random, ratio=(.7,.2,.1),
                   group_prefix = None) #splited into train test an validation, can be splited into only 2 

In [32]:
directory = 'Splited_Data_Sample/'
files = ['train','val','test']
for file in files:
    for filename in os.listdir(directory+file):
        for img in os.listdir(directory+file+"/"+filename):
            full_path = directory+file+"/"+filename+"/"+img
            with Image.open(full_path) as im:
                im.save(full_path, "png")
    else: continue